In [76]:
import pandas as pd

import time
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader,Dataset
from torch import nn
import pennylane as qml

from IPython.display import Image

In [77]:
df = pd.read_parquet('solar_radiation_hourly_rangamati_10_years.parquet')
df = df.dropna(how='any')
df.set_index('Time')
df.Time = df.Time.astype('int')
df

,Time,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,TS,RH2M,PS,WS50M,SZA,PRECTOTCORR,WS10M,QV2M
0,2010010106,10.45,8.15,0.36,308.02,0.00,12.01,12.41,12.46,100.00,101.04,4.05,88.82,0.0,2.75,8.91
1,2010010107,139.84,148.05,0.50,312.48,0.20,13.69,12.87,15.45,94.69,101.15,3.92,78.62,0.0,3.04,9.16
2,2010010108,334.69,359.02,0.62,328.77,1.20,15.49,13.36,18.74,87.00,101.23,3.58,67.30,0.0,3.17,9.46
3,2010010109,514.27,544.30,0.68,335.90,2.85,18.38,12.94,21.64,70.50,101.28,4.59,57.47,0.0,4.01,9.22
4,2010010110,685.15,679.58,0.76,313.05,4.73,20.09,13.10,23.79,64.00,101.25,4.70,50.00,0.0,4.18,9.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87643,2020010101,0.00,0.00,-999.00,335.52,0.00,18.11,15.55,17.01,84.88,101.33,2.84,-999.00,0.0,2.04,10.93
87644,2020010102,0.00,0.00,-999.00,336.88,0.00,17.85,15.60,16.90,86.50,101.30,3.01,-999.00,0.0,2.13,10.93
87645,2020010103,0.00,0.00,-999.00,335.52,0.00,17.73,15.69,16.84,87.69,101.27,2.99,-999.00,0.0,2.12,10.99
87646,2020010104,0.00,0.00,-999.00,334.70,0.00,17.63,15.68,16.54,88.25,101.28,2.77,-999.00,0.0,2.00,10.99


In [78]:
df = df.drop(index=df[(df == -999).any(axis=1)].index)
df

,Time,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,TS,RH2M,PS,WS50M,SZA,PRECTOTCORR,WS10M,QV2M
0,2010010106,10.45,8.15,0.36,308.02,0.00,12.01,12.41,12.46,100.00,101.04,4.05,88.82,0.0,2.75,8.91
1,2010010107,139.84,148.05,0.50,312.48,0.20,13.69,12.87,15.45,94.69,101.15,3.92,78.62,0.0,3.04,9.16
2,2010010108,334.69,359.02,0.62,328.77,1.20,15.49,13.36,18.74,87.00,101.23,3.58,67.30,0.0,3.17,9.46
3,2010010109,514.27,544.30,0.68,335.90,2.85,18.38,12.94,21.64,70.50,101.28,4.59,57.47,0.0,4.01,9.22
4,2010010110,685.15,679.58,0.76,313.05,4.73,20.09,13.10,23.79,64.00,101.25,4.70,50.00,0.0,4.18,9.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87630,2019123112,719.53,729.92,0.74,338.50,6.35,25.24,17.07,28.80,60.44,101.30,1.01,46.70,0.0,0.80,12.02
87631,2019123113,654.30,649.42,0.75,337.52,4.95,25.55,17.01,28.12,59.19,101.18,1.27,51.54,0.0,1.09,12.02
87632,2019123114,502.70,501.85,0.70,343.65,2.80,25.50,16.90,26.87,58.94,101.11,1.37,59.64,0.0,1.20,11.90
87633,2019123115,289.05,308.88,0.59,349.83,0.95,25.29,16.92,25.40,59.81,101.09,1.32,69.86,0.0,1.14,11.96


In [79]:
df=df.drop(columns=['CLRSKY_SFC_SW_DWN','ALLSKY_SFC_LW_DWN'])
df

,Time,ALLSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,TS,RH2M,PS,WS50M,SZA,PRECTOTCORR,WS10M,QV2M
0,2010010106,10.45,0.36,0.00,12.01,12.41,12.46,100.00,101.04,4.05,88.82,0.0,2.75,8.91
1,2010010107,139.84,0.50,0.20,13.69,12.87,15.45,94.69,101.15,3.92,78.62,0.0,3.04,9.16
2,2010010108,334.69,0.62,1.20,15.49,13.36,18.74,87.00,101.23,3.58,67.30,0.0,3.17,9.46
3,2010010109,514.27,0.68,2.85,18.38,12.94,21.64,70.50,101.28,4.59,57.47,0.0,4.01,9.22
4,2010010110,685.15,0.76,4.73,20.09,13.10,23.79,64.00,101.25,4.70,50.00,0.0,4.18,9.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87630,2019123112,719.53,0.74,6.35,25.24,17.07,28.80,60.44,101.30,1.01,46.70,0.0,0.80,12.02
87631,2019123113,654.30,0.75,4.95,25.55,17.01,28.12,59.19,101.18,1.27,51.54,0.0,1.09,12.02
87632,2019123114,502.70,0.70,2.80,25.50,16.90,26.87,58.94,101.11,1.37,59.64,0.0,1.20,11.90
87633,2019123115,289.05,0.59,0.95,25.29,16.92,25.40,59.81,101.09,1.32,69.86,0.0,1.14,11.96


In [80]:
target = 'ALLSKY_SFC_SW_DWN'
features = list(df.columns.difference([target]))

In [81]:
features

['ALLSKY_KT',
 'ALLSKY_SFC_UV_INDEX',
 'PRECTOTCORR',
 'PS',
 'QV2M',
 'RH2M',
 'SZA',
 'T2M',
 'T2MDEW',
 'TS',
 'Time',
 'WS10M',
 'WS50M']

In [82]:
df = df.reset_index(drop=True)

size = int(len(df) * 0.7)

df_train = df.loc[:size].copy()
df_test = df.loc[size:].copy()

df_train

,Time,ALLSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,TS,RH2M,PS,WS50M,SZA,PRECTOTCORR,WS10M,QV2M
0,2010010106,10.45,0.36,0.00,12.01,12.41,12.46,100.00,101.04,4.05,88.82,0.0,2.75,8.91
1,2010010107,139.84,0.50,0.20,13.69,12.87,15.45,94.69,101.15,3.92,78.62,0.0,3.04,9.16
2,2010010108,334.69,0.62,1.20,15.49,13.36,18.74,87.00,101.23,3.58,67.30,0.0,3.17,9.46
3,2010010109,514.27,0.68,2.85,18.38,12.94,21.64,70.50,101.28,4.59,57.47,0.0,4.01,9.22
4,2010010110,685.15,0.76,4.73,20.09,13.10,23.79,64.00,101.25,4.70,50.00,0.0,4.18,9.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31988,2016123013,598.53,0.69,4.65,24.44,18.01,27.44,67.25,101.08,2.40,51.57,0.0,2.25,12.76
31989,2016123014,428.91,0.60,2.75,24.29,17.89,26.30,67.38,101.02,2.33,59.67,0.0,2.20,12.70
31990,2016123015,269.48,0.55,1.02,23.82,17.76,24.51,68.75,101.00,2.24,69.90,0.0,2.05,12.63
31991,2016123016,96.29,0.45,0.10,22.76,18.36,22.15,76.12,101.02,2.45,81.19,0.0,1.84,13.06


In [83]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

In [84]:
df_train

,Time,ALLSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,TS,RH2M,PS,WS50M,SZA,PRECTOTCORR,WS10M,QV2M
0,-1.528173,-1.391395,-0.230802,-1.081792,-3.813981,-1.619096,-3.395484,1.621582,1.206771,-0.122214,1.551947,-0.370569,-0.440105,-1.569430
1,-1.528173,-0.903903,0.059228,-1.019015,-3.397121,-1.534553,-2.788846,1.319852,1.419734,-0.179298,1.103261,-0.370569,-0.283807,-1.517843
2,-1.528172,-0.169783,0.307825,-0.705131,-2.950485,-1.444498,-2.121342,0.882883,1.574617,-0.328594,0.605307,-0.370569,-0.213743,-1.455939
3,-1.528172,0.506806,0.432123,-0.187222,-2.233388,-1.521688,-1.532964,-0.054695,1.671418,0.114903,0.172896,-0.370569,0.238981,-1.505462
4,-1.528171,1.150616,0.597855,0.402881,-1.809084,-1.492282,-1.096753,-0.424045,1.613337,0.163205,-0.155700,-0.370569,0.330603,-1.493081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31988,1.528364,0.824265,0.452840,0.377770,-0.729715,-0.589887,-0.356209,-0.239370,1.284212,-0.846740,-0.086638,-0.370569,-0.709583,-0.774987
31989,1.528364,0.185202,0.266392,-0.218610,-0.766935,-0.611942,-0.587502,-0.231983,1.168050,-0.877477,0.269672,-0.370569,-0.736531,-0.787368
31990,1.528365,-0.415469,0.162810,-0.761630,-0.883556,-0.635834,-0.950673,-0.154136,1.129330,-0.916997,0.719678,-0.370569,-0.817375,-0.801812
31991,1.528365,-1.067982,-0.044355,-1.050403,-1.146575,-0.525562,-1.429491,0.264650,1.168050,-0.824784,1.216312,-0.370569,-0.930556,-0.713082


In [85]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[self.target].values).float()
        self.X = torch.tensor(dataframe[self.features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]


In [86]:
torch.manual_seed(101)

batch_size = 1
sequence_length = 3

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)



Features shape: torch.Size([1, 3, 13])
Target shape: torch.Size([1])


In [87]:
from torch import nn

class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [88]:
learning_rate = 5e-5
num_hidden_units = 16

model = ShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [89]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")


print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

for ix_epoch in range(2):
    print(f"Epoch {ix_epoch}\n---------")
    train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_model(test_loader, model, loss_function)
    print()

Untrained test
--------
Test loss: 1.1438315515730604

Epoch 0
---------
Train loss: 0.1366961191851238
Test loss: 0.023441905987207134

Epoch 1
---------
Train loss: 0.009961429600893353
Test loss: 0.012175098846057535



In [91]:
def predict(data_loader, model):

    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    
    return output


train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
df_test[ystar_col] = predict(test_loader, model).numpy()

df_out = df_test[[target, ystar_col]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean

print(df_out)

       ALLSKY_SFC_SW_DWN  Model forecast
31992               9.24       34.341370
31993             107.77      113.443115
31994             267.44      261.246704
31995             402.91      391.988617
31996             534.34      507.440887
...                  ...             ...
45698             719.53      686.954102
45699             654.30      619.321960
45700             502.70      472.199127
45701             289.05      290.596832
45702             101.40      128.388245

[13711 rows x 2 columns]


/tmp/ipykernel_11944/1458429284.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out[c] = df_out[c] * target_stdev + target_mean
